# Unit commitment problem II - Thermal Power Station

Inspired by example from book Einführung in Optimirungsmodelle from Nathan Sudermann-Merx page 127

Problem statement:

> Every hour, a Thermal power station must decide how much energy to produce. Demand for energy is known upfront and has to be respected. Station has a maximum production capacity of 500 MW and a minimum production capacity of 100 MW. The station can ramp up or down its production by at most 120 MW per hour. If the station is turned off, it must stay off for at least 8 hours. The variable production cost is 40 EUR/MWh and the start-up cost is 50,000 EUR. The goal is to minimize the total cost of production while respecting the demand and operational constraints.

In [23]:
import polars as pl
import pyoframe as pf

demand = pl.read_csv('demand.csv').with_row_index('hours')
demand


hours,demand
u32,f64
0,310.0
1,322.352381
2,401.5
3,424.388348
4,455.253175
…,…
19,206.259272
20,265.746825
21,189.611652


Minimum and maximum production of the power plant

In [24]:
P_max = 500
P_min = 100

Maximum ramping rate

In [25]:
delta_max = 120

Minimum downtime

In [26]:
L_min = 8

Variable production costs

In [27]:
c = 40

Start-up costs

In [28]:
S = 50000

Optimization model

In [29]:
m = pf.Model("unit commitment problem", solver='highs', use_var_names=True)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


Decision variables:

Power production at each hour, `P`, is a continuous variable with upper bound `P_max`.

In [30]:
m.P = pf.Variable(demand[['hours']], lb=0, ub=P_max)

on/off status of the power plant at each hour, `Y`, is a binary variable.

In [31]:
m.Y = pf.Variable(demand[['hours']], vtype=pf.VType.BINARY)

on switch

In [32]:
m.Delta_on = pf.Variable(demand[['hours']], vtype=pf.VType.BINARY)#%% md

off switch

In [33]:
m.Delta_off = pf.Variable(demand[['hours']], vtype=pf.VType.BINARY)

Constraints:

Fix the status variable for the first hour to 1 (the plant is on at the beginning).

In [34]:
m.initial_status = m.Y.filter(pl.col('hours') == demand['hours'].min()) == 1

Cover the demand at each hour with the power production.

In [35]:
m.cover_the_demand = m.P >= demand

The power production must be within the limits of the power plant when plant is on.

In [36]:
m.production_limits = P_min * m.Y <= m.P <= m.Y * P_max

The power plant cannot be switched on and off at the same hour.

NOTE: check if this is needed at all. Yes it is needed, otherwise the link_status_and_switches could have both Delta_on and Delta_off equal to 1 at the same hour while stil satisfying the link_status_and_switches constraint. But since we have minimisation problem starting and stopping in the same hour would certanly increase the cost so it is not optimal.

In [37]:
m.turn_on_and_off_exclusive = m.Delta_on + m.Delta_off <= 1

Link the on/off status of the power plant and turning the switch from on to off and vice versa.

In [46]:
m.link_status_and_switches = (
        m.Y.next('hours') - m.Y.drop_unmatched()
    == m.Delta_on.next('hours') - m.Delta_off.next('hours')
)

Minimal downtime constraint: if the plant is turned on, it must stay on for at least `L_min` hours.

In [ ]:
L_min * m.Delta_on <= (1 - m.Y).rolling_sum()

Objective function: minimize the total cost, which is the sum of variable production costs and start-up costs.

In [47]:
m.minimize = pf.sum(
    c * m.P + S * m.Delta_on)

Hessian has dimension 97 but no nonzeros, so is ignored


In [48]:
m.optimize()

MIP  has 96 rows; 97 cols; 262 nonzeros; 72 integer variables (72 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [4e+01, 5e+04]
  Bound  [1e+00, 5e+02]
  RHS    [0e+00, 0e+00]
Presolving model
93 rows, 93 cols, 207 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   307040          307040             0.00%        0      0      0    

In [50]:
m.P.solution

hours,solution
u32,f64
0,310.0
1,322.352381
2,401.5
3,424.388348
4,455.253175
…,…
19,206.259272
20,265.746825
21,189.611652
